# Introducción a _word embeddings_ y _fastText_

### Parte 1 de 3 sobre **_fastText_**
### 19/06/2019

## 1: Introducción

Los _word embeddings_ son **una representación vectorial densa, distribuida y de longitud fija construida a partir del cómputo de coocurrencias siguiendo la hipótesis distribucional** (Almeida y Xexéo, 2019), y surgen como una estrategia para representar palabras y documentos que permita aplicar, de manera intuitiva, diferentes operaciones sobre la información codificada. Hay dos partes de importancia en esta definición: por un lado, tenemos la cuestión sobre la **representación vectorial densa, distribuida y de longitud fija**, y por el otro, el hecho de que se trate de una construcción a partir de **la distribución de coocurrencias**.

### 1.1: Las representaciones vectoriales

El uso de representaciones vectoriales para operar sobre documentos proviene del modelo de espacio vectorial, que se originó en la comunidad de extracción de información, y que en su forma más básica, el modelo de bolsa de palabras, supone que un documento puede representarse como un recuento de la cantidad de ocurrencias de los distintos términos que contiene, y un conjunto de documentos se representa como una matriz cuyas filas corresponden a los documentos y cuyas columnas son las palabras presentes en todos los documentos (Manning, Raghavan, y Schütze, 2008).

![title](imagenes_y_bibliografia/documentterm_matrix.jpg)

*Imagen extraída de https://www.listendata.com/2015/09/text-mining-basics.html*

Estas representaciones pueden complejizarse más, emplear valores distintos en los valores de la matriz, como en el esquema _tf-idf_. La virtud fundamental de las representaciones vectoriales es que permiten operar algebraicamente (Axler, 2015) y que estas operaciones se pueden comprender de manera más o menos intuitiva: si sumamos dos vectores, podemos suponer que estamos sumando la información que representan; si calculamos el producto escalar, podemos suponer que estamos cuantificando el grado en el que un vector difiere respecto del otro.

![title](imagenes_y_bibliografia/similitud_manning.jpg)

*Imagen extraída de Manning et al. (2008)*

En un modelo de espacio vectorial, cada una de las columnas de la matriz codifica una porción de información distinta, lo que quiere decir que la información codificada está **distribuida** en el vector. Asimismo, dentro de un modelo de espacio vectorial, todos los documentos tienen un vector de la misma dimensión, con lo cual la representación posee una **longitud fija**. Sin embargo, en esta instancia la mayoría de los valores de la matriz va a ser cero, lo cual es una consecuencia trivial de la ley de Zipf: en una colección de documentos, la gran mayoría de los términos va a aparecer una cantidad reducida de veces, por lo tanto también va a aparecer en una cantidad reducida de documentos, y eso quiere decir que para la mayoría de los documentos el valor de esa columna va a ser cero. Este tipo de matrices se llaman **matrices dispersas** (_sparse matrices_), y poseen el defecto de que, a medida que aumenta la cantidad de documentos en el corpus, aumenta muchísimo la dimensión de la matriz. Lo que desearíamos, entonces, es tener una representación que codifique la misma información que estas matrices pero en una cantidad de dimensiones más reducida y que la totalidad de los valores del vector sea informativa, en vez de tener múltiples dimensiones con valor nulo. Esta propiedad es la **densidad** de la representación.


### 1.2: La hipótesis distribucional

Prácticamente todos los trabajos sobre representaciones vectoriales incluyen una referencia a Firth (1957) que dice que uno puede conocer el significado de una palabra partiendo de las palabras que suelen acompañarla. Un desarrollo que toma esta hipótesis es el de los modelos estadísticos del lenguaje. Esta familia de modelos supone, por ejemplo, que puede predecirse la palabra siguiente en un texto a partir del contexto precedente. Distintos modelos de lenguaje van a construir el contexto de diferentes maneras. El modelo más básico, el modelo de unigramas, va a suponer que no existe el contexto y que la posibilidad de una palabra es la probabilidad correspondiente a esa palabra, donde la probabilidad corresponde al porcentaje de veces que esa palabra aparece en el corpus. Un modelo de _n-gramas_ va a suponer que la posibilidad de estimar la palabra siguiente es su probabilidad condicional dadas las **n - 1** palabras precedentes.

Distintos modelos van a implementar diferentes definiciones de contexto y de probabilidad, pero comparten el supuesto fundamental de que el significado de cada palabra corresponde a **la distribución de las palabras que aparecen en el mismo contexto**.


### 1.3: Tipos de _embeddings_

La definición de _word embeddings_, entonces, constituye una amalgama de los dos puntos previos: las palabras se representan a través de vectores que se generan a partir de sus distribuciones de frecuencia dentro de un contexto determinado. Además, considera la cuestión de las representaciones **densas**: se toman matrices dispersas y se las incrusta (_embed_) en un espacio de menor dimensión.

Los _embeddings_ se pueden clasificar en dos tipos, según la metodología que emplean para la construcción de las matrices dispersas cuya dimensión se reduce: hay modelos basados en conteo, que emplean modelos de bolsa de palabras y derivados, y modelos predictivos, que toman como base modelos de _n-gramas_. Los modelos de conteo son más antiguos. El primer algoritmo de este tipo es **LSA/LSI** (_latent semantic analysis/indexing_) (Dumais, Furnas, Landauer, Deerwester, y Harshman, 1988), de principios de los 90. Hubo numerosos desarrollos posteriores, y el estado del arte actual en este tipo de modelos es **GloVe** (Pennington, Socher, y Manning, 2014). Los modelos predictivos son el resultado de la construcción de modelos de n-gramas a través de redes neuronales. La primer propuesta de este tipo de modelo es la de Bengio, Ducharme, Vincent, y Jauvin (2003), que fue sucesivamente refinada. El inicio del estándar actual, sin embargo, es el sistema **_word2vec_**, propuesto por  Mikolov, Sutskever, Chen, Corrado, y Dean (2013), que consiste en dos algoritmos, **_skip-gram_** y **CBOW** (_continuous bag of words_). **_fastText_**  (Bojanowski, Grave, Joulin, y Mikolov, 2016; Joulin, Grave, Bojanowski, y Mikolov, 2016)  es un desarrollo posterior sobre _word2vec_ en el que las matrices iniciales no son sólo de predicción sobre _n-gramas_ de _tokens_, sino que además toma _n-gramas_ de caracteres, lo cual le permite inferir _embeddings_ para términos no presentes en el corpus de entrenamiento y además permite capturar regularidades formales entre los _tokens_, tratando de hacer palabras que formen parte de un mismo paradigma flexivo o derivativo tengan posiciones cercanas en el espacio vectorial.

Para situaciones reales de uso, en general se considera que _fastText_ » _GloVe_ » _word2vec_.

# 2: Las arquitecturas _skip-gram_ y CBOW

Los dos modelos que componen _word2vec_ (y que _fastText_ hereda) suponen el entrenamiento de una red neuronal de una única capa oculta, y constituyen formas complementarias de realizar predicciones sobre un texto. La arquitectura _skip-gram_ consiste en la predicción del contexto dada una palabra determinada, mientras que CBOW es la tarea inversa, inferir una palabra dado su contexto.


### 2.1: _Skip-gram_

Esta arquitectura supone una red neuronal con una capa de entrada de dimensión V igual al tamaño del vocabulario, una capa oculta de tamaño $\textbf{N}$, donde $\textbf{N}$ es el tamaño que prefijemos para los _embeddings_ y una capa de salida de dimensión $\textbf{CxV}$, donde $\textbf{C}$ es el tamaño del contexto, definido como la cantidad de palabras alrededor de la palabra de entrada. Así, el input de un modelo _skip-gram_ es una única palabra **w<sub>input</sub>**, representada por un vector _one-hot_ de dimensión $\textbf{V}$ con valor 0 en todas sus dimensiones excepto la que corresponde a su posición en el vocabulario y cuyo valor es 1. El ouput del modelo consiste en las palabras en el contexto de **w<sub>input</sub>**, **{w<sub>output,-C</sub>, ..., w<sub>output,C</sub>}**. Así, para **w<sub>input</sub>** = $incoloras$ y $\textbf{C}$ = 2, el output podría consistir en los vectores _one-hot_ correspondientes a $\{ideas, verdes, duermen, furiosamente\}$. Entre la capa de input y la capa oculta se incluye la matriz de pesos $\textbf{W}$ de dimensión $\textbf{VxN}$ y entre la capa oculta y la capa de output hay una matriz de pesos $\textbf{W'}$ de dimensión $\textbf{NxV}$.

![title](imagenes_y_bibliografia/skip-gram.png)

*Imagen extraída de http://mccormickml.com/assets/word2vec/Alex_Minnaar_Word2Vec_Tutorial_Part_I_The_Skip-Gram_Model.pdf)*

En esta configuración, el valor de la capa oculta correspondiente a cada palabra equivale a seleccionar la fila correspondiente en la matriz $\textbf{W}$. El texto empleado para entrenar un modelo de _embeddings_ provee varias instancias de **w<sub>input</sub>** y de **{w<sub>output,-C</sub>, ..., w<sub>output,C</sub>}** que permiten actualizar iterativamente los pesos de la matriz $\textbf{W}$. El resultado final del entrenamiento son los pesos de $\textbf{W}$, y el vector de cada palabra es su fila correpondiente en la matriz.

### 2.2: CBOW

La arquitectura CBOW es prácticamente una imagen especular de _skip-gram_. En la capa de input se hallan las palabras en una ventana contextual $\textbf{C}$, en el output el vector _one-hot_ correspondiente a la palabra que se intenta predecir **w<sub>output</sub>**.

![title](imagenes_y_bibliografia/cbow.png)

*Imagen extraída de http://mccormickml.com/assets/word2vec/Alex_Minnaar_Word2Vec_Tutorial_Part_II_The_Continuous_Bag-of-Words_Model.pdf*




En este caso, los _embeddings_ correspondientes a cada palabra son los valores de la matriz $\textbf{W'}$.

### 2.3: _Embeddings_ de caracteres

La diferencia más importante entre _word2vec_ y _fastText_ radica en cómo se construyen los embeddings de cada palabra. Mientras que para _word2vec_ las palabras se representan como vectores _one-hot_ de dimensión igual al tamaño del vocabulario, en _fastText_ se determinan tamaños mínimos y maximos de _n-gramas_ de caracteres **n<sub>min</sub>** y **n<sub>max</sub>**. Para **n<sub>min</sub>** = 2 y **n<sub>max</sub>** = 4, la representación de $incoloras$ es la suma de los vectores _one-hot_ de $\{<i, in, nc, co, ol, lo, or, ra, as, s>, <in, inc, nco, col, olo, lor, ora, ras, as>, <inc, inco, ncol, colo, olor, lora, oras, ras>\}$, junto con la secuencia $<incoloras>$, donde los caracteres $\{<, >\}$ representan el comienzo y el final de un token. 

# 3: Implementación

_fastText_ puede emplearse desde la línea de comandos o como una librería de python. En estos encuentros vamos a usar solamente la implementación de python. Para ver más sobre ambas implementaciones, pueden consultar Bhattacharjee (2018)


### 3.1 Entrenamiento

La aplicación permite entrenar modelos de _word embeddings_ para emplear en otras tareas de PLN o un clasificador supervisado que entrena _embeddings_ y los usa como _features_ para distinguir entre un conjunto predefinido de categorías. 

In [ ]:
import fastText as ft
help(ft.train_unsupervised)

El modelo no supervisado toma los siguientes parámetros:

- input: Un archivo de texto plano. Cada documento debe estar separado por '\n', caso contrario se trunca a los 1024 tokens.
- model: 'skipgram' o 'cbow'.
- lr: Tasa de aprendizaje, indica la cantidad máxima de variación en los pesos del modelo ante cada iteración de entrenamiento. Un valor de 0 significa que los pesos no se modifican. Se recomienda un valor entre 0.1 y 1.
- dim: La dimensión deseada de los _embeddings_.
- ws: Tamaño de la ventana contextual.
- epoch: Cantidad de veces que el modelo ve cada ejemplo durante el entrenamiento.
- minCount: Cantidad mínima de veces que un token debe aparecer para ser incluido en el vocabulario.
- minCountLabel: [Sólo lo usamos en el caso supervisado].
- minn: Tamaño mínimo de los n-gramas de caracteres.
- maxn: Tamaño máximo de los n-gramas de caracteres.
- neg: Tamaño del muestreo negativo. Tiene que ver con el modo en el que se optimiza la actualización de los pesos, no me voy a meter en esto pero pueden leer al respecto acá: http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/
- wordNgrams: Tamaño de los n-gramas de palabras.
- loss: Función de costo.
- bucket: Cantidad de subgrupos en los que se divide el vocabulario.
- thread: Cantidad de cadenas en las que se paraleliza el entrenamiento.
- lrUpdateRate: Velocidad a la que los pesos decaen a cero (se usa para evitar _overfitting_).
- t: Umbral para descartar palabras sobrerrepresentadas. Sólo funciona en el caso no supervisado.
- label: [Sólo lo usamos en el caso supervisado].
- verbose: Cantidad de información que se proporciona durante el entrenamiento. No se imprime en las celdas de la notebook, pero sí en el terminal desde el que se ejecuta jupyter.
- pretrainedVectors: Archivo .vec con información de los vectores entrenados en otro modelo, se recomienda su uso cuando el corpus de entrenamiento es pequeño.

In [ ]:
help(ft.train_supervised)

El modelo no supervisado tiene un requisito adicional sobre el archivo de input que es que cada línea debe contener un token de forma $PREFIJO$ + $CLASE$ (ej: $\_label\_POSITIVO$), que _fastText_ interpreta como la etiqueta correspondiente a la línea. La función de entrenamiento toma casi todos los argumentos del caso no supervisado, excepto que no permite seleccionar entre _skip-gram_ y CBOW, los defaults son diferentes, y sí funcionan los siguientes parámetros:

- minCountLabel: Cantidad mínima de veces que debe aparecer una clase para ser considerada por el clasificador.
- label: Prefijo que debe llevar un token para ser identificado como una etiqueta de clase.

Ahora vamos a entrenar dos modelos, uno no supervisado y uno supervisado.

#### 3.1.1: Modelo no supervisado

Para poder realizar el entrenamiento hace falta juntar todos los textos en un único archivo. Por lo pronto mantuve la puntuación, pero separé el texto en una oración por línea para evitar el truncamiento.

In [ ]:
import os
import re
import numpy as np
import pandas as pd
from string import punctuation
from nltk.tokenize import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity


punct = re.compile(r'[{}¡¿…«»—]'.format(punctuation))


def bulk_read(filedir):
    for file in os.listdir(filedir):
        with open('{}/{}'.format(filedir, file)) as data:
            item = ' '.join(data.readlines())
        yield item
        
        
def keep_pattern(pattern):
    return ' {} '.format(pattern.group())


def space_punct(line):
    return punct.sub(keep_pattern, line)


def prepare_texts(texts):
    for text in texts:
        sentences = sent_tokenize(text)
        for sentence in sentences:
            prepared = space_punct(sentence)        
            yield prepared


In [ ]:
with open('train_noSupervisado.txt', 'w+') as newfile:
    for line in prepare_texts(bulk_read('../bestiario')):
        newfile.write('{}\n'.format(line))

Acá pueden ver que elegí algunos parámetros que difieren de los que tiene _fastText_ por defecto.

In [ ]:
modelo_noSup = ft.train_unsupervised(
    input = 'train_noSupervisado.txt',
    dim = 150,
    ws = 2,
    epoch = 10,
    minCount = 2,
    wordNgrams = 3
)

Podemos consultar el vocabulario del modelo.

In [ ]:
vocab = modelo_noSup.get_words()
print(vocab[:20])

Consultar el vector correspondiente a una palabra en particular.

In [ ]:
Buenos = modelo_noSup.get_word_vector('Buenos')
print(Buenos.shape)
Buenos

Si queremos consultar las distancias entre todos los términos, vamos a tener que recurrir a alguna otra herramienta, ya que los bindings de _fastText_ para python no incluyen una función propia para consultar distancias o vecinos.

In [ ]:
def make_distanceFrame(modelo):
    vocabulario = modelo.get_words()
    matrix = np.array(
        [modelo_noSup.get_word_vector(x) for x in modelo_noSup.get_words()]
    )
    distancias = pd.DataFrame(
        cosine_similarity(matrix),
        columns = vocabulario, index = vocabulario
    )
    return distancias


def get_neighbors(distanceFrame, word, num = 20):
    neighbors = distanceFrame[word].sort_values(ascending = False).head(num)
    return neighbors


def get_distance(distanceFrame, word1, word2):
    distance = distanceFrame[word1][word2]
    return distance

In [ ]:
unsup_distances = make_distanceFrame(modelo_noSup)

In [ ]:
get_distance(unsup_distances, 'Buenos', 'Aires')

In [ ]:
get_neighbors(unsup_distances, 'Buenos')

Los modelos ocupan una buena cantidad de espacio en memoria, así que vamos a guardar el modelo y eliminar las variables que contienen información.

In [ ]:
modelo_noSup.save_model('CortazarNoSup.bin')
del(modelo_noSup, unsup_distances)

#### 3.1.2: Modelo supervisado

Ahora vamos a entrenar un modelo supervisado. Para eso vamos a agarrar tanto los textos de Cortázar como los de Borges y etiquetarlos según el autor.
También vamos a hacer una partición train/test para evaluarlo.

In [ ]:
from random import sample, seed

cortazar = ['__label__CORTAZAR\t{}'.format(line) for line in prepare_texts(bulk_read('../bestiario'))]
borges = ['__label__BORGES\t{}'.format(line) for line in prepare_texts(bulk_read('../el_aleph'))]
ordenado = cortazar + borges

seed(1)
randomizado = sample(ordenado, len(ordenado))

train = randomizado[:int(.9 * len(randomizado))]
test = randomizado[len(train):]

In [ ]:
with open('train_Supervisado.txt', 'w+') as newfile:
    for line in train:
        newfile.write('{}\n'.format(line))
        
with open('test_Supervisado.txt', 'w+') as newfile:
    for line in test:
        newfile.write('{}\n'.format(line))

In [ ]:
modelo_sup = ft.train_supervised('train_Supervisado.txt')

Para evaluar el modelo, podemos usar el método _test_ sobre el archivo correspondiente. Esto devuelve la cantidad de ejemplos, _precision_ y _recall_

In [ ]:
modelo_sup.test('test_Supervisado.txt')

También podemos usar el modelo para predecir la etiqueta correspondiente a una linea. Para eso, sin embargo, tenemos que abrir el archivo y ejecutar el método _predict_ sobre cada línea.

In [ ]:
with open('test_Supervisado.txt') as test:
    lines = test.readlines()
    
for line in lines[:20]:
    print(line, modelo_sup.predict(line[:-1]))

Como vemos, el método _predict_ devuelve la etiqueta más probable y la probabilidad que se le asigna. Uno podría pedirle al modelo que devuelva las $\textbf{N}$ etiquetas más probables.

In [ ]:
for line in lines[20:40]:
    print(line, modelo_sup.predict(line[:-1], 2))

Ante la duda, se pueden consultar cuáles son todas las clases que aparecen en el modelo.

In [ ]:
modelo_sup.get_labels()

Finalmente, volvemos a eliminar el modelo de la memoria.

In [ ]:
modelo_sup.save_model('CortazarXBorgesSup.bin')
del(modelo_sup)

# 4: Otras APIs

Más allá de las herramientas propias del paquete oficial, hay otras opciones para trabajar con _fastText_. El paquete **skift** proporciona un _wrapper_ que se integra con la API de **sklearn** y facilita usar **pandas** para entrenar modelos. El paquete **gensim** también tiene su propio _wrapper_ que permite un acceso a los modelos de _fastText_ similar al de los demás algoritmos incluidos en el paquete. En las próximas reuniones vamos a ver cómo usar estas opciones.

## Referencias

- Almeida, F., y Xexéo, G. (2019). Word embeddings: A survey. _CoRR, abs/1901.09069_.
- Axler, S. J. (2015). _Linear algebra done right_ (3<sup>ra</sup> ed.). Springer International Publishing.
- Bengio, Y., Ducharme, R., Vincent, P., y Jauvin, C. (2003). A neural probabilistic language model. JMLR, 1137–1155.
- Bhattacharjee, J. (2018). _fasttext quick start guide: Get started with facebook’s library for text representation and classification_ (1<sup>ra</sup> ed.). Packt Publishing.
- Bojanowski, P., Grave, E., Joulin, A., y Mikolov, T. (2016). Enriching word vectors with subword information. _CoRR, abs/1607.04606_.
- Dumais, S. T., Furnas, G. W., Landauer, T. K., Deerwester, S., y Harshman, R. (1988). Using latent semantic analysis to improve access to textual information. En _Proceedings of the sigchi conference on human factors in computing systems(pp. 281–285)_.
- Firth, J. R. (1957). Applications of general linguistics. _Transactions of the Philological Society_, 56(1), 1-14.
- Joulin, A., Grave, E., Bojanowski, P., y Mikolov, T. (2016). Bag of tricks for efficient text classification. _arXiv preprint arXiv:1607.01759_.
- Manning, C. D., Raghavan, P., y Schütze, H. (2008). _Introduction to information retrieval_. Cambridge, UK: Cambridge University Press.
- Mikolov, T., Sutskever, I., Chen, K., Corrado, G. S., y Dean, J. (2013). Distributed representations of words and  phrases and their compositionality. En _Advances in neural information processing systems_ (pp. 3111–3119).
- Pennington, J., Socher, R., y Manning, C. D. (2014). Glove: Global vectors for word repre-sentation.

## Sitios web

- https://fasttext.cc/
- http://alexminnaar.com/2015/04/12/word2vec-tutorial-skipgram.html
- http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
- http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/
- http://mccormickml.com/assets/word2vec/Alex_Minnaar_Word2Vec_Tutorial_Part_II_The_Continuous_Bag-of-Words_Model.pdf
- https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/
- https://towardsdatascience.com/fasttext-under-the-hood-11efc57b2b3
